In [5]:
datasetName = "a51724fd-8fcd-4e0b-a8f0-38d0539b81c5"

syntheticDataPath = "synthetic_dataset/temp"

imageWidth = 640
imageHeight = 640

In [6]:
import os

datasetPath = f"synthetic_dataset/{datasetName}"

metadataName = next(filter(lambda x: x.startswith("Dataset"), os.listdir(datasetPath)))
metadataPath = f"{datasetPath}/{metadataName}"

captureFiles = list(filter(lambda x: x.startswith("captures_"), os.listdir(metadataPath)))

In [7]:
import json
import shutil
from uuid import uuid4

class_count = {
    "Black": 0,
    "Brown": 0,
    "Gray": 0,
    "White": 0,
}

shutil.rmtree(syntheticDataPath, ignore_errors=True, onerror=None)
os.makedirs(syntheticDataPath)

for captureFile in captureFiles:
    with open(f"{metadataPath}/{captureFile}") as f:
        data = json.load(f)
    
    for capture in data["captures"]:
        fileName = f"{datasetPath}/{capture['filename']}"
        annotations = capture["annotations"][0]["values"]
        if len(annotations) < 1:
            continue
        
        image_name = uuid4().hex
        
        shutil.copyfile(fileName, f"{syntheticDataPath}/{image_name}.png")

        with open(f"{syntheticDataPath}/{image_name}.txt", "w+") as f:
            for label in annotations:
                class_name = label["label_name"]
                class_id = label["label_id"]
                class_count[class_name] += 1

                x = label["x"] / imageWidth
                y = label["y"] / imageHeight
                w = label["width"] / imageWidth
                h = label["height"] / imageHeight

                xc = x + w / 2
                yc = y + h / 2

                if w > 2e-2 and h > 2e-2:
                    f.write(f"{class_id} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}\n")


In [8]:
class_count

{'Black': 1516, 'Brown': 2039, 'Gray': 1051, 'White': 566}

# Remove empty images

In [11]:
import os

syntheticDataPath = "synthetic_dataset/temp"

images = list(filter(lambda x: x.endswith(".png"), os.listdir(syntheticDataPath)))
labels = list(filter(lambda x: x.endswith(".txt"), os.listdir(syntheticDataPath)))

for image in images:
    name = image[:-4]
    if f"{name}.txt" not in labels:
        os.remove(f"{syntheticDataPath}/{image}")

for label in labels:
    with open(f"{syntheticDataPath}/{label}", "r") as f:
        if f.readline() == "":
            name = label[:-4]
            os.remove(f"{syntheticDataPath}/{label}")
            os.remove(f"{syntheticDataPath}/{name}.png")

# Stats

In [12]:
import os

syntheticDataPath = "synthetic_dataset/temp"

classes = ["n/a", "Black", "Brown", "Gray", "White"]

class_count = {
    "Black": 0,
    "Brown": 0,
    "Gray": 0,
    "White": 0,
}

labels = list(filter(lambda x: x.endswith(".txt"), os.listdir(syntheticDataPath)))

for label in labels:
    with open(f"{syntheticDataPath}/{label}", "r") as f:
        while (line := f.readline()) != "":
            clas = int(line[0])
            class_count[classes[clas]] += 1

In [13]:
class_count

{'Black': 1206, 'Brown': 1583, 'Gray': 812, 'White': 459}

In [14]:
class_count["Brown"] + class_count["Black"] + class_count["Gray"] + class_count["White"]

4060

In [15]:
len(labels)

1940

# Move labels

In [16]:
import os
import shutil

syntheticDataPath = "synthetic_dataset/temp"
syntheticImagePath = "grid_dataset/synthetic/images"
syntheticLabelPath = "grid_dataset/synthetic/labels"

images = list(filter(lambda x: x.endswith(".png"), os.listdir(syntheticDataPath)))
labels = list(filter(lambda x: x.endswith(".txt"), os.listdir(syntheticDataPath)))

for label in labels:
    shutil.move(f"{syntheticDataPath}/{label}", f"{syntheticLabelPath}/{label}")

for image in images:
    shutil.move(f"{syntheticDataPath}/{label}", f"{syntheticImagePath}/{image}")